# A Learning Algorithm for Boltzmann Machines

http://www.cs.toronto.edu/~fritz/absps/cogscibm.pdf

    DAVID H. ACKLEY 
    GEOFFREY E. HINTON
    TERRENCE J. SEJNOWSKI
    
    COGNITIVE SCIENCE 9, 147-169(1985)

## 总结
1. 每个unit只有两种状态：0和1，表示accept/reject
1. 边的权重值是实数，且可正可负,正表示support，负则相反
1. 每个global state被指定为一个值“energy”，整个网络的目标是最小化这个global energy
1. 模拟退火：跳出局部最小值点
1. Boltzmann分布：某两个最终状态的概率比与它们的global state有关
1. "encoder problem":检验信息损失问题

## THE BOLTZMANN MACHINE
1. The `machine` is composed of primitive computing elements called `units` that are connected to each other by bidirectional `links`.
1. A `unit` is always in one of two `states`, `on` or `off`, and it adopts these states as a probabilistic function of the states of its neighboring units and the weights on its links to them. 
1. The `weights` can take on real values of either `sign`. 
1. A `unit` being on or off is taken to mean that the system currently accepts or rejects some elemental hypothesis about the domain. 
1. The `weight` on a link **represents a weak pairwise constraint between two hypotheses**(这里hypotheses指的是前后两层的两个unit). A positive weight indicates that the two hypotheses tend to support one another; if one is currently accepted, accepting the other should be more likely. Conversely, a negative weight suggests,other things being equal, that the two hypotheses should not both be accepted. 
1. Link `weights` are symmetric, having the same strength in both directions (Hinton & Sejnowski, 1983).

The resulting structure is related to a system described by Hopfield (1982), and as in his system, each `global state` of the network can be assigned a single number called the “`energy`” of that state. (每一个 `global state` 都用`energy`来量化)With the right assumptions, the individual units can be made to act so as to **minimize the global energy**. If some of the units are externally forced or “clamped” into particular states to represent a particular input, **the system will then find the minimum energy configuration that is compatible with that input**(调整内部系统，以最小代价满足，这个调整的代价是`global energy`). 

The energy of a global configuration is defined as

$E=-\sum_{i < j}w_{ij}s_is_j+\sum_i \theta_is_i \ (1)$

- $w_{ij}$ is the strength of connection between units i and j
- $s_i$ is 1 if unit i is on and 0 otherwise
- $\theta_i$ is a threshold
 
### Minimizing Energy
A simple algorithm for finding a combination of truth values that is a `local minimum` is to switch each hypothesis into whichever of its two states yields the lower total energy given the current states of the other hypotheses. If hardware units make their decisions asynchronously, and if transmission times are negligible, then the system always settles into a local energy minimum (Hopfield, 1982). Because the connections are symmetric, the difference between the energy of the whole system with the kth hypothesis rejected and its energy with the kth hypothesis accepted can be determined locally by the kth unit, and this `energy gap` is just(从单点角度去观察问题)

$\triangle E_k=\sum_i w_{ki}s_i-\theta_k \ (2)$

Therefore, **the rule for minimizing the energy contributed by a unit is to adopt the on state if its total input from the other units and from outside the system exceeds its threshold.**

The threshold terms can be eliminated from Eqs.(1) and (2) by making the following observation:the effect of $\theta_i$ on the global energy or on the energy gap of an individual unit is identical to the effect of a link with strength $-\theta_i$ between unit i and a special unit that is by definition always held in the on state. This "true unit" need have no physical reality, but it simplifies the computations by **allowing the threshold of a unit to be treated in the same manner as the links**.(把threshold当成一个unit看待)

The value $-\theta_i$ is called the `bias` of unit i.

(1) and (2) can be written as:

$E=-\sum_{i<j}w_{ij}s_is_j\ (3)$

$\triangle E_k=\sum_i w_{ki}s_i\ (4)$

### Using Noise to Escape from Local Minima
It gets stuck in local minima that are not globally optimal.

A simple way to get out of local minima is to occasionally allow jumps to configurations of higher energy.

If the energy gap between the on and off states of the kth unit is $\triangle E_k$ then regardless of the previous state set $s_k=1$ with probability

$p_k=\frac{1}{(1+e^{-\triangle E_k/T})}\ (5)$

where T is a parameter that acts like temperature (see Figure 1).

![1](http://ou8qjsj0m.bkt.clouddn.com//17-8-6/90005256.jpg)

The decision rule in Eq. (5) is the same as that for a particle which has two energy states. A system of such particles in contact with a heat bath at a given temperature will eventually reach thermal equilibrium and the probability of finding the system in any global state will then obey a `Boltzmann distribution`. Similarly, a network of units obeying this decision rule will eventually reach “thermal equilibrium” and the relative probability of two global states will follow the Boltzman distribution:

$\frac{P_\alpha}{P_\beta}=e^{-(E_\alpha-E_\beta)/T}\ (6)$

where $P_\alpha$, is the probability of being in the $\alpha^{th}$ global state, and $E_\alpha$, is the energy of that state.

The Boltzmann distribution has some beautiful mathematical properties and it is intimately related to information theory. In particular, the difference in the log probabilities of two global states is just their energy difference (at a temperature of 1).

## A LEARNING ALGORITHM
Perhaps the most interesting aspect of the Boltzmann Machine formulation is that it **leads to a domain-independent learning algorithm that modifies the connection strengths between units in such a way that the whole network develops an internal model which captures the underlying structure of its environment.**

The `“credit-assignment” problem`:The major technical stumbling block which prevented the generalization of simple learning algorithms to more complex networks was this: To be capable of interesting computations, a network must contain nonlinear elements that are not directly constrained by the input, and when such a network does the wrong thing it appears to be impossible to decide which of the many connection strengths is at fault.

Solved within the Boltzmann Machine formulation: By using the right stochastic decision rule, and by running the network until it reaches “thermal equilibrium” at some finite temperature, we achieve a mathematically simple relationship between the probability of a global state and its energy.

For a network that is running freely without any input from the environment, this relationship is given by Eq. (6). Because the energy is a linear function of the weights (Eq. 1) this leads to a remarkably simple relationship between the log probabilities of global states and the individual connection strengths:

$\frac{\partial In P_{\alpha}}{\partial w_{ij}}=\frac{1}{T}[s_i^{\alpha}s_j^{\alpha}-p_{ij}^{'}]\ (7)$

- $s_i^{\alpha}$ is the state of the $i^{th}$ unit in the $\alpha^{th}$ global state(so $s_i^{\alpha}s_j^{\alpha}$ is 1 only if units i and j are both on in state $\alpha$)
- $p_{ij}^{'}$ is just the probability of finding the two units i and j on at the same time when the system is at equilibrium.

Given Eq. (7). it is possible to manipulate the log probabilities of global states. If the environment directly specifies the required probabilities $P_{\alpha}$, for each global state $\alpha$, there is a straightforward way of converging on a set of weights that achieve those probabilities,provided any such set exists (for details, see Hinton & Sejnowski, 1983a). 

 However, this is not a particularly interesting kind of learning because the system has to be given the required probabilities of **complete** global states. This means that the central question of what internal representation should be used has already been decided by the environment. The interesting problem arises when the environment implicitly contains high-order constraints and the network must choose internal representations that allow these constraints to be expressed efficiently.

### Modeling the Underlying Structure of an Environment
The units of a Boltzmann Machine partition into two functional groups, a nonempty set of `visible` units and a possibly empty set of `hidden` units.

An information-theoretic measure of the discrepancy between the network’s internal model and the environment is

$G=\sum_{\alpha}P(V_{\alpha})In\frac{P(V_{\alpha})}{P^{'}(V_{\alpha})}\ (8)$

- $P(V_{\alpha})$ is the probability of the $\alpha^{th}$ state of the visiable units when their state are determined by the envirnment.
- $P^{'}(V_{\alpha})$ is the corresponding probability when the network is running freely with no environmental input.
- G metric, sometimes called the asymmetric divergence or `information gain` (Kullback, 1959; Renyi, 1962), is a measure of the distance from the distribution given by the $P^{'}(V_{\alpha})$ to the distribution given by the $P(V_{\alpha})$. G is zero if and only if the distributions are identical; otherwise it is positive.

The term $P^{'}(V_{\alpha})$ depends on the weights, and so G can be altered by changing them. To perform gradient descent in G, it is necessary to know the partial derivative of G with respect to each individual weight.

The probabilities of global states are determined by their energies (Eq. 6) and the energies are determined by the weights (Eq. 1). Using these equations the partial derivative of G (see the appendix) is:

$\frac{\partial G}{\partial w_{ij}}=-\frac{1}{T}(p_{ij}-p_{ij}^{'})\ (9)$

- $p_{ij}$ is the average probability of two units both being in the on state when the environment is clamping the states of the visible units,
- $p_{ij}^{'}$, as in Eq. (7), is the corresponding probability when the environmental input is not present and the network is running freely. 
- Both these probabilities must be measured at equilibrium.
- Note the similarity between this equation and Eq. (7), which shows how changing a weight affects the log probability of a single state.

To minimize G, it is therefore sufficient to observe $p_{ij}$, and $p_{ij}^{'}$; when the network is at thermal equilibrium, and to change each weight by an amount proportional to the difference between these two probabilities:

$\Delta w_{ij}=\epsilon(p_{ij}-p_{ij}^{'})\ (10)$

- $\epsilon$ scales the size of each weight change.

Once G has been minimized the network will have captured as well as possible the regularities in the environment, and these regularities will be enforced when performing completion. An alternative view is that the network, in minimizing G, is finding the set of weights that is most likely to have generated the set of environmental vectors. It can be shown that maximizing this likelihood is mathematically equivalent to minimizing G (Peter Brown, personal communication, 1983).

### Controlling the Learning
The objective function G is a metric that specifieshow well two probability distributions match.

## THE ENCODER PROBLEM
We have used this problem to test out the learning algorithm because it is clear what the optimal solution is like and it is nontrivial to discover it. Two groups of visible units, designated $V_1$, and $V_2$, represent two systems that wish to communicate their states. Each group has v units. In the simple formulation we consider here, each group has only one unit on at a time, so there are only v different states of each group. $V_1$, and $V_2$ are not connected directly but both are connected to a group of h hidden units H, with h < v so H may act as a limited capacity bottleneck through which information about the states of $V_1$, and $V_2$ **must be squeezed**. Since all simulations began with all weights set to zero, finding a solution to such a problem requires that the two visible groups come to agree upon the meanings of a set of codes without any apriori conventions for communication through H.

###  The 4-2-4 Encoder
1. Estimation of $p_{ij}$: Each environmental vector in turn was clamped over the visible units. For each environmental vector, the network was allowed to reach equilibrium twice. Statistics about how often pairs of units were both on together were gathered at equilibrium. To prevent the weights from growing too large we used the “noisy” clamping technique described in Section 3.2. Each on bit of a clamped vector was set to off with a probability of 0.15 and each off bit was set to on with a probability of 0.05.
2. Estimation of $p_{ij}^{'}$: The network was completely unclamped and allowed to reach equilibrium at a temperature of 10. Statistics about co-occurrences were then gathered for as many annealings as were used to estimate $p_{ij}$.
3. Updating the weighs: All weights in the network were incremented or decremented by a fixed weight-step of 2, with the sign of the increment being determined by the sign of $p_{ij}-p_{ij}^{'}$.
